Loading Data

In [ ]:
 %load_ext kedro.ipython

In [ ]:
catalog.list()

In [ ]:
data = catalog.load("bangladesh_pois_json@local")

In [ ]:
counter = 1
for i, d in enumerate(data):
    counter += i
    x = d
    break

In [ ]:
import h3
import numpy as np
import pandas as pd
poibatch = x

In [ ]:
#utils
def enbn_from_name(name):
    enlist = None
    bnlist = None
    name_translate_en2bn = None
    name_translate_bn2en = None
    if isinstance(name, str):
        extracted_bn_from_name = []
        extracted_en_from_name = []
        words = name.split(" ")
        for word in words:
            if bool(regex.fullmatch(r'\P{L}*\p{Bengali}+(?:\P{L}+\p{Bengali}+)*\P{L}*', word)):
                extracted_bn_from_name.append(word)
            else:
                extracted_en_from_name.append(word)
        if extracted_en_from_name: 
            enlist = " ".join(extracted_en_from_name)
        if extracted_bn_from_name: 
            bnlist = " ".join(extracted_bn_from_name)
        if extracted_en_from_name and not extracted_bn_from_name:
            name_translate_en2bn = " ".join(extracted_en_from_name)
        if not extracted_en_from_name and extracted_bn_from_name:
            name_translate_bn2en = " ".join(extracted_bn_from_name)
    return enlist, bnlist, name_translate_en2bn, name_translate_bn2en

#utils
def enbn_from_parts(addr_dictionary):
    separator = GLOBAL_SEPARATOR
    enlist = []
    bnlist = []
    name_translate_en2bn = None
    name_translate_bn2en = None
    for part in addr_dictionary.get("parts"):
        if part and part.get("names", None): 
            name_keys = list(part.get("names").keys())
            for tag, value in part.get("names").items():
                extracted_bn_from_name = []
                extracted_en_from_name = []
                if tag == "name":
                    words = value.split(" ")
                    for word in words:
                        if bool(regex.fullmatch(r'\P{L}*\p{Bengali}+(?:\P{L}+\p{Bengali}+)*\P{L}*', word)):
                            extracted_bn_from_name.append(word)
                        else:
                            extracted_en_from_name.append(word)
                    if extracted_en_from_name: enlist.append(" ".join(extracted_en_from_name))
                    if extracted_bn_from_name: bnlist.append(" ".join(extracted_bn_from_name))
                    if extracted_en_from_name and not extracted_bn_from_name:
                        if part.get("lvl") in ['street'] and "name:bn" not in name_keys:
                            name_translate_en2bn = " ".join(extracted_en_from_name)
                    if not extracted_en_from_name and extracted_bn_from_name:
                        if part.get("lvl") in ['street'] and "name:en" not in name_keys:
                            name_translate_bn2en = " ".join(extracted_bn_from_name)
                if tag == "name:en":
                    enlist.append(value)
                if tag == "name:bn":
                    bnlist.append(value)
        else:
            return np.nan, np.nan, name_translate_en2bn, name_translate_bn2en
    enlist = list(dict.fromkeys(enlist))    
    bnlist = list(dict.fromkeys(bnlist))
    return separator.join(enlist), separator.join(bnlist), name_translate_en2bn, name_translate_bn2en

In [ ]:
poi_columns = ["osm_id", "h3level9", "lat", "lon", "name", "name_en", "name_bn", 
              "need_translate_en2bn", "need_translate_bn2en",
              "alt_names", "local_admin_name", "admin0_name", "admin2_name", 
              "addr_level", "poi_addr_match", "poi_class", "poi_keywords"]
h3_level = 9
GLOBAL_SEPARATOR = ", "
separator = GLOBAL_SEPARATOR

In [ ]:
z = "সোহাগ বুকস এন্ড স্টেশনারি sohag books"

In [ ]:
enbn_from_name(z)

In [ ]:
poibatch[~poibatch["need_translate_en2bn_extension"].isna()]["need_translate_en2bn_extension"]

In [ ]:
poibatch["need_translate_en2bn_short"].fillna("") 

In [ ]:
poibatch["need_translate_en2bn_short"].fillna("") + separator + poibatch["need_translate_en2bn_extension"].fillna("")

In [ ]:
poibatch[~poibatch["need_translate_bn2en_short"].isna()]["need_translate_bn2en_short"]

In [ ]:
poibatch["h3level9"] = poibatch["center_point"].apply(lambda x: h3.geo_to_h3(lat=x.get('lat'), lng=x.get('lon'), resolution=h3_level))
poibatch["lat"] = poibatch["center_point"].apply(lambda x: x.get('lat'))
poibatch["lon"] = poibatch["center_point"].apply(lambda x: x.get('lon'))
poibatch["name_en_short"], poibatch["name_bn_short"], poibatch["need_translate_en2bn_short"], poibatch["need_translate_bn2en_short"] \
        = zip(*poibatch["name"].apply(lambda x: enbn_from_name(x)))
poibatch["name_en_extension"], poibatch["name_bn_extension"], poibatch["need_translate_en2bn_extension"], poibatch["need_translate_bn2en_extension"] \
    = zip(*poibatch["address"].apply(lambda x: enbn_from_parts(x)))

poibatch["name_en"] = (poibatch["name_en_short"].fillna("") + separator + poibatch["name_en_extension"]).str.lstrip(separator).fillna('')
poibatch["name_bn"] = (poibatch["name_bn_short"].fillna("") + separator + poibatch["name_bn_extension"]).str.lstrip(separator).fillna('')
poibatch["name_en"] = poibatch["name_en"].str.split(separator).apply(lambda x: separator.join(list(dict.fromkeys(x))))
poibatch["name_bn"] = poibatch["name_bn"].str.split(separator).apply(lambda x: separator.join(list(dict.fromkeys(x))))

poibatch["need_translate_en2bn"] = (poibatch["need_translate_en2bn_short"].fillna("") + separator + poibatch["need_translate_en2bn_extension"].fillna("")).str.lstrip(separator).fillna('')
poibatch["need_translate_bn2en"] = (poibatch["need_translate_bn2en_short"].fillna("") + separator + poibatch["need_translate_bn2en_extension"].fillna("")).str.lstrip(separator).fillna('')
poibatch["need_translate_en2bn"] = poibatch["need_translate_en2bn"].str.split(separator).apply(lambda x: separator.join(list(dict.fromkeys(x))))
poibatch["need_translate_bn2en"] = poibatch["need_translate_bn2en"].str.split(separator).apply(lambda x: separator.join(list(dict.fromkeys(x))))

# found_columns = set(poibatch.columns)
# missing_columns = set(poi_columns).union(found_columns) - found_columns
# if missing_columns:
#     for col in missing_columns:
#         poibatch[col] = np.nan

In [ ]:
poibatch[["name", "name_en", "name_bn", "need_translate_bn2en", "need_translate_en2bn"]]

In [ ]:
poibatch[~poibatch["need_translate_bn2en_extension"].isna()]

In [ ]:
x.columns

In [ ]:
import h3
x["h3level9"] = x["center_point"].apply(lambda x: h3.geo_to_h3(lat=x.get('lat'), lng=x.get('lon'), resolution=9))

In [ ]:
x["lat"] = x["center_point"].apply(lambda x: x.get('lat'))
x["lon"] = x["center_point"].apply(lambda x: x.get('lon'))

In [ ]:
import math
import regex
import numpy as np
def enbn_from_parts(addr_dictionary):
    enlist = []
    bnlist = []
    name_translate_en2bn = None
    name_translate_bn2en = None
    for part in addr_dictionary.get("parts"):
        if part and part.get("names", None): 
            name_keys = list(part.get("names").keys())
            for tag, value in part.get("names").items():
                extracted_bn_from_name = []
                extracted_en_from_name = []
                if tag == "name":
                    words = value.split(" ")
                    for word in words:
                        if bool(regex.fullmatch(r'\P{L}*\p{Bengali}+(?:\P{L}+\p{Bengali}+)*\P{L}*', word)):
                            extracted_bn_from_name.append(word)
                        else:
                            extracted_en_from_name.append(word)
                    if extracted_en_from_name: enlist.append(" ".join(extracted_en_from_name))
                    if extracted_bn_from_name: bnlist.append(" ".join(extracted_bn_from_name))
                    if extracted_en_from_name and not extracted_bn_from_name:
                        if part.get("lvl") in ['street'] and "name:bn" not in name_keys:
                            name_translate_en2bn = " ".join(extracted_en_from_name)
                    if not extracted_en_from_name and extracted_bn_from_name:
                        if part.get("lvl") in ['street'] and "name:en" not in name_keys:
                            name_translate_bn2en = " ".join(extracted_bn_from_name)
                if tag == "name:en":
                    enlist.append(value)
                if tag == "name:bn":
                    bnlist.append(value)
    enlist = list(dict.fromkeys(enlist))    
    bnlist = list(dict.fromkeys(bnlist))
    return ", ".join(enlist), ", ".join(bnlist), name_translate_en2bn, name_translate_bn2en
#     return enlist, bnlist, name_translate_en2bn, name_translate_bn2en

In [ ]:
def enbn_from_name(name):
    enlist = None
    bnlist = None
    name_translate_en2bn = None
    name_translate_bn2en = None
    if isinstance(name, str):
        extracted_bn_from_name = []
        extracted_en_from_name = []
        words = name.split(" ")
        for word in words:
            if bool(regex.fullmatch(r'\P{L}*\p{Bengali}+(?:\P{L}+\p{Bengali}+)*\P{L}*', word)):
                extracted_bn_from_name.append(word)
            else:
                extracted_en_from_name.append(word)
        if extracted_en_from_name: 
            enlist = " ".join(extracted_en_from_name)
        if extracted_bn_from_name: 
            bnlist = " ".join(extracted_bn_from_name)
        if extracted_en_from_name and not extracted_bn_from_name:
            name_translate_en2bn = " ".join(extracted_en_from_name)
        if not extracted_en_from_name and extracted_bn_from_name:
            name_translate_bn2en = " ".join(extracted_bn_from_name)
    return enlist, bnlist, name_translate_en2bn, name_translate_bn2en

In [ ]:
x["name_en_short"], x["name_bn_short"], x["need_translate_en2bn_short"], x["need_translate_bn2en_short"] \
    = zip(*x["name"].apply(lambda x: enbn_from_name(x)))

In [ ]:
x["name_en_extension"], x["name_bn_extension"], x["need_translate_en2bn_extension"], x["need_translate_bn2en_extension"] \
    = zip(*x["address"].apply(lambda x: enbn_from_parts(x)))

In [ ]:
x["name_en"] = (x["name_en_short"].fillna("") + ", " + x["name_en_extension"]).str.lstrip(',')
x["name_bn"] = (x["name_bn_short"].fillna("") + ", " + x["name_bn_extension"]).str.lstrip(',')

In [ ]:
x["name_en"] = x["name_en"].str.split(", ").apply(lambda x: ", ".join(list(dict.fromkeys(x))))
x["name_bn"] = x["name_bn"].str.split(", ").apply(lambda x: ", ".join(list(dict.fromkeys(x))))

In [ ]:
x["need_translate_en2bn"] = (x["need_translate_en2bn_short"].fillna("") + "," + x["need_translate_en2bn_extension"]).str.lstrip(',')
x["need_translate_bn2en"] = (x["need_translate_bn2en_short"].fillna("") + "," + x["need_translate_bn2en_extension"]).str.lstrip(',')

In [ ]:
# x[x["need_translate_en2bn"].notna()]

In [ ]:
poi_info = x[["osm_id", "h3level9", "lat", "lon", "name", 
#               "name_en", "name_bn", "need_translate_en2bn", "need_translate_bn2en",
              "name_en_short", "name_bn_short", "need_translate_en2bn_short", "need_translate_bn2en_short",
              "name_en_extension", "name_bn_extension", "need_translate_en2bn_extension", "need_translate_bn2en_extension",
              "alt_names", "local_admin_name", "admin0_name", "admin2_name", 
              "addr_level", "poi_addr_match", "poi_class", "poi_keywords"]]

In [ ]:
poi_info

Cleaning Data

In [ ]:
gadm_data = catalog.load("geojson_bangladesh@gpd")
gadm_data

In [ ]:
import geopandas as gpd

In [ ]:
bdl0 = gadm_data['gadm41_BGD_0.json']()
bdl1 = gadm_data['gadm41_BGD_1.json']()
bdl2 = gadm_data['gadm41_BGD_2.json']()
bdl3 = gadm_data['gadm41_BGD_3.json']()
bdl4 = gadm_data['gadm41_BGD_4.json']()

In [ ]:
bdl4

In [ ]:
import geopandas as gpd
import h3pandas

# h3.polyfill_polygon(

In [ ]:
gdf = bdl4[["geometry"]]
gdf_h3l9 = gdf.h3.polyfill(9)
f = gdf_h3l9.explode("h3_polyfill")
len(f["h3_polyfill"].unique())

In [ ]:
gdf = bdl0[["geometry"]]
gdf_h3l9 = gdf.h3.polyfill(9)
f = gdf_h3l9.explode("h3_polyfill")
len(f["h3_polyfill"].unique())

In [ ]:
((1221635 - 1217895)/1221635)*100

In [ ]:
bdl420 = bdl4[["COUNTRY", "NAME_1", "NAME_2", "NAME_3", "NAME_4"]]
bdl420.columns = ["country", "division", "district", "thana", "union"]

In [ ]:
gdf = bdl4[["geometry"]]
gdf_h3l9 = gdf.h3.polyfill(9)

In [ ]:
bdl420

In [ ]:
req_h3_data = gdf_h3l9[["h3_polyfill"]].join(bdl420)

In [ ]:
req_h3_data

In [ ]:
import hashlib

def md5hash(s: str): 
    return hashlib.md5(s.encode('utf-8')).hexdigest()

req_h3_data["region_hash"] = req_h3_data.apply(lambda x: md5hash("-".join((x["country"], x["division"], x["district"], x["thana"], x["union"]))), axis = 1)

In [ ]:
len(req_h3_data["region_hash"].unique())

In [ ]:
req_h3_data.rename(columns={"h3_polyfill": "h3level9"}, inplace=True)

In [ ]:
req_h3_data = req_h3_data.explode("h3level9")

In [ ]:
len(req_h3_data["h3level9"].unique())

In [ ]:
req_h3_data.info()

In [ ]:
req_h3_data[req_h3_data["h3level9"].isna()]

In [ ]:
req_h3_data[req_h3_data["region_hash"].eq("22f2fc16bb22466c8679b40650c491ff")]

In [ ]:
poi_info

In [ ]:
final_data = poi_info.merge(req_h3_data, how="left", on="h3level9")

In [ ]:
final_data["district"] = final_data["district"].fillna('foo')

In [ ]:
final_data

In [ ]:
groups = final_data.groupby(by="district")
for k, g in groups:
#     g.to_csv(f'{k}.csv')
    print(k)
    

In [ ]:
a = set([1,2,3,5])

In [ ]:
b = set([1,2,3])

In [ ]:
a.union(b) - b

In [ ]:
import pandas as pd
import numpy as np
path = "D:\\MY_REPOSITORY\\kedro_101\\_101\\data\\02_intermediate\\bdh3\\csv\\poi\\Dhaka.csv".replace("\\", "/")

In [ ]:
path

In [ ]:
foo = pd.read_csv(path)

In [ ]:
foo

In [ ]:
foo[~foo["need_translate_bn2en"].isna()]

In [ ]:
foo

In [ ]:
if set():
    print(1)